**Table of contents**<a id='toc0_'></a>    
- [Prompt template](#toc1_)    
  - [Output parser](#toc1_1_)    
- [Retrieval Chain](#toc2_)    
- [Conversation Retrieval Chain](#toc3_)    
- [Agent](#toc4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()  # take environment variables from .env.

api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key=api_key)

In [3]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing by providing automated testing tools and frameworks that can be used to test software applications. These tools can help automate the testing process, making it faster and more efficient. Langsmith can also provide support for writing test cases, executing tests, and analyzing test results. Additionally, Langsmith can help with setting up continuous integration and continuous deployment pipelines to automate the testing and deployment process. Overall, Langsmith can help improve the quality of software applications by streamlining the testing process and ensuring that software is thoroughly tested before being released.', response_metadata={'token_usage': {'completion_tokens': 109, 'prompt_tokens': 15, 'total_tokens': 124}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

# <a id='toc1_'></a>[Prompt template](#toc0_)

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are world class technical documentation writer."),
        ("user", "{input}"),
    ]
)

chain = prompt | llm

In [5]:
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content='Langsmith is a powerful tool that can greatly assist with testing activities. Here are some ways Langsmith can help with testing:\n\n1. Automated testing: Langsmith can be used to automate testing processes, reducing the need for manual testing and increasing test coverage.\n\n2. Test data generation: Langsmith can generate synthetic test data to simulate various scenarios and edge cases, helping to ensure comprehensive testing coverage.\n\n3. Test case generation: Langsmith can automatically generate test cases based on specifications or requirements, saving time and effort in creating test cases manually.\n\n4. Integration testing: Langsmith can be used to test the integration of different components or systems, helping to identify potential issues early in the development process.\n\n5. Performance testing: Langsmith can simulate high loads and stress conditions to test the performance of an application or system under various scenarios.\n\n6. Regression testing: 

## <a id='toc1_1_'></a>[Output parser](#toc0_)

In [6]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [7]:
chain = prompt | llm | output_parser

In [8]:
chain.invoke({"input": "how can langsmith help with testing?"})

'Langsmith can help with testing in several ways:\n\n1. Automated Testing: Langsmith can be used to generate test cases automatically based on the specifications of the language. This can help in quickly creating a comprehensive set of test cases to cover various scenarios.\n\n2. Test Data Generation: Langsmith can be used to generate test data that can be used to validate the functionality of a system. This can help in ensuring that the system behaves as expected under different input conditions.\n\n3. Test Coverage Analysis: Langsmith can be used to analyze the coverage of test cases generated for a system. This can help in identifying areas of the system that are not adequately covered by the tests, allowing for more focused testing efforts.\n\n4. Test Oracles: Langsmith can be used to create test oracles that can be used to determine the expected output of a system given a particular input. This can help in automating the process of verifying the correctness of the system.\n\nOvera

# <a id='toc2_'></a>[Retrieval Chain](#toc0_)

```bash
pip install beautifulsoup4
```

In [9]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

Next, we need to index it into a vectorstore. This requires a few components, namely an [embedding model](https://python.langchain.com/docs/modules/data_connection/text_embedding) and a [vectorstore](https://python.langchain.com/docs/modules/data_connection/vectorstores).

In [10]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

Now, we can use this embedding model to ingest documents into a vectorstore. We will use a simple local vectorstore, [FAISS](https://faiss.ai/), for simplicity's sake.

```bash
pip install faiss-cpu
```

In [11]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

First, let's set up the chain that takes a question and the retrieved documents and generates an answer.

In [12]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}"""
)

document_chain = create_stuff_documents_chain(llm, prompt)

we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [13]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [14]:
response = retrieval_chain.invoke(
    {"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

LangSmith can help with testing by allowing developers to create datasets, run tests on LLM applications, upload test cases in bulk, create custom evaluations, compare results for different configurations, use a playground environment for rapid iteration and experimentation, collect feedback from users, annotate traces, add runs to datasets, and monitor key metrics over time.


# <a id='toc3_'></a>[Conversation Retrieval Chain](#toc0_)

In [15]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        (
            "user",
            "Given the above conversation, generate a search query to look up to get information relevant to the conversation",
        ),
    ]
)
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [16]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(content="Can LangSmith help test my LLM applications?"),
    AIMessage(content="Yes!"),
]
retriever_chain.invoke({"chat_history": chat_history, "input": "Tell me how"})

[Document(page_content='Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phase.Debugging\u200bWhen developing new LLM appli

In [17]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
    ]
)
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [18]:
chat_history = [
    HumanMessage(content="Can LangSmith help test my LLM applications?"),
    AIMessage(content="Yes!"),
]
retrieval_chain.invoke({"chat_history": chat_history, "input": "Tell me how"})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly u

# <a id='toc4_'></a>[Agent](#toc0_)

In [19]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [20]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [21]:
tools = [retriever_tool, search]

In [23]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# You need to set OPENAI_API_KEY environment variable or pass it as argument `openai_api_key`.
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [26]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping​Prototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.
The ability to rapidly understand how the model is performing — and debug where i

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing in various ways throughout the application development lifecycle. Here are some key ways LangSmith supports testing:\n\n1. Prototyping: LangSmith allows for quick experimentation between prompts, model types, retrieval strategies, and other parameters to understand how the model is performing during the prototyping phase.\n\n2. Debugging: LangSmith provides tracing capabilities that give clear visibility and debugging information at each step of an LLM sequence, making it easier to identify and root-cause issues when things go wrong.\n\n3. Initial Test Set: Developers can create datasets of inputs and reference outputs to run tests on LLM applications. LangSmith makes it easy to run custom evaluations to score test results.\n\n4. Comparison View: LangSmith offers a user-friendly comparison view for test runs to track and diagnose regressions in test scores across multiple revisions of an appli

In [27]:
agent_executor.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://weather.com/weather/hourbyhour/l/USCA0987:1:US', 'content': "recents\nSpecialty Forecasts\nHourly Weather-San Francisco, CA\nBeach Hazard Statement\nSaturday, November 25\n5 pm\nClear\n6 pm\nClear\n7 pm\nClear\n8 pm\nMostly Clear\n9 pm\nPartly Cloudy\n10 pm\nPartly Cloudy\n11 pm\nPartly Cloudy\nSunday, November 26\n12 am\nPartly Cloudy\n1 am\nMostly Cloudy\n2 am\nMostly Cloudy\n3 am\nMostly Cloudy\n4 am\nCloudy\n5 am\nCloudy\n6 am\nMostly Cloudy\n7 am\nMostly Cloudy\n8 am\nMostly Cloudy\n9 am\nMostly Cloudy\n10 am\nPartly Cloudy\n11 am\nPartly Cloudy\n12 pm\nPartly Cloudy\n1 pm\nPartly Cloudy\n2 pm\nPartly Cloudy\n3 pm\nPartly Cloudy\n4 pm\nPartly Cloudy\n5 pm\nPartly Cloudy\n6 pm\nPartly Cloudy\n7 pm\nPartly Cloudy\n8 pm\nPartly Cloudy\n9 pm\nMostly Clear\n10 pm\nMostly Clear\n11 pm\nMostly Clear\nMonday, November 27\n12 am\nClear\n1 am\nMost

{'input': 'what is the weather in SF?',
 'output': 'The weather in San Francisco is currently clear with temperatures ranging from mostly clear to partly cloudy throughout the day and night. You can find more detailed hourly weather updates on [weather.com](https://weather.com/weather/hourbyhour/l/USCA0987:1:US).'}

In [28]:
agent_executor.invoke({"input": "Summary of the MLB game last night?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'MLB game summary last night'}`


[{'url': 'https://www.espn.com/mlb/', 'content': "BREAKING DOWN THE SOTO TRADE\nSLUGGER HEADING TO THE BRONX\nJuan Soto is headed to the Yankees -- but the Padres might have won the trade\nHow much the 25-year-old slugger helps the Yankees -- and why the Padres still might have gotten the better end of the deal.\n ESPN\nQuick Links\nFavorites\nCustomize ESPN\nPodcasts\nESPN Sites\nESPN Apps\nFollow ESPN\nOhtani's new contract is just his latest feat to shock the world\nFor six years, he has amazed us on the field. Top Headlines\nFavorites\nFantasy\nCustomize ESPN\nBest of ESPN+\nHow Yankees (and Juan Soto!), Mets can change their fortunes this winter\n Let's debate the fallout of Ohtani's eye-popping record deal with Dodgers\nOhtani is headed to the other side of L.A. on a contract that blows away every other in the history of the sport.\n BIGGEST MOVES THIS

{'input': 'Summary of the MLB game last night?',
 'output': 'I found information about the MLB game last night. The Houston Astros defeated the Texas Rangers 8-5 in Game 3 of the American League Championship Series. The Rangers now lead the best-of-seven series by a 2-1 margin.'}